# 08. XGBoost & LightGBM

## 학습 목표
- Gradient Boosting 개념 이해
- XGBoost 사용법과 하이퍼파라미터
- LightGBM 특징과 최적화
- CatBoost 개요
- 모델 비교 및 선택

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
from sklearn.datasets import make_classification, load_breast_cancer, fetch_california_housing
import time

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

## 1. Gradient Boosting 개념

In [ ]:
print("""
Gradient Boosting 알고리즘:

1. 초기화: F_0(x) = argmin_γ Σ L(y_i, γ)

2. 반복 (m = 1, 2, ..., M):
   a. 의사 잔차(pseudo-residual) 계산:
      r_im = -[∂L(y_i, F(x_i))/∂F(x_i)]_{F=F_{m-1}}
   
   b. 잔차에 대해 약한 학습기 h_m(x) 학습
   
   c. 최적 스텝 크기 계산:
      γ_m = argmin_γ Σ L(y_i, F_{m-1}(x_i) + γ * h_m(x_i))
   
   d. 모델 업데이트:
      F_m(x) = F_{m-1}(x) + learning_rate * γ_m * h_m(x)

핵심:
- 각 단계에서 이전 모델의 오차(잔차)를 학습
- 손실 함수의 그래디언트 방향으로 최적화
- learning_rate로 과적합 방지
""")

# 간단한 시각화 데이터
np.random.seed(42)
X_demo = np.linspace(0, 10, 100).reshape(-1, 1)
y_demo = np.sin(X_demo).ravel() + np.random.randn(100) * 0.3

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.scatter(X_demo, y_demo, alpha=0.5)
plt.xlabel('X')
plt.ylabel('y')
plt.title('Sample Data for Gradient Boosting')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
stages = [0, 1, 5, 20, 50]
colors = ['red', 'orange', 'yellow', 'green', 'blue']
for stage, color in zip(stages, colors):
    if stage == 0:
        plt.axhline(y=np.mean(y_demo), color=color, label=f'Stage {stage}', alpha=0.7)
plt.scatter(X_demo, y_demo, alpha=0.3, color='gray')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Gradient Boosting: Sequential Learning')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. XGBoost (eXtreme Gradient Boosting)

In [ ]:
# XGBoost 설치: pip install xgboost
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor

print("""
XGBoost 특징:

1. 정규화:
   - L1, L2 정규화로 과적합 방지
   - 목표 함수: Σ L(y_i, ŷ_i) + Σ Ω(f_k)
   - Ω(f) = γT + 0.5λ||w||²

2. 효율적인 계산:
   - 2차 테일러 전개 사용
   - 히스토그램 기반 분할
   - 캐시 최적화

3. 결측치 처리:
   - 자동으로 최적 방향 학습

4. 병렬 처리:
   - 특성별 병렬 분할점 탐색
""")

print(f"XGBoost 버전: {xgb.__version__}")

### 2.1 XGBoost 분류

In [ ]:
# Breast Cancer 데이터 로드
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"데이터 크기: {X.shape}")
print(f"클래스: {cancer.target_names}")

In [ ]:
# XGBoost 분류기
xgb_clf = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    min_child_weight=1,     # 리프 노드 최소 가중치
    gamma=0,                # 분할에 필요한 최소 손실 감소
    subsample=1.0,          # 행 샘플링 비율
    colsample_bytree=1.0,   # 트리별 열 샘플링 비율
    reg_alpha=0,            # L1 정규화
    reg_lambda=1,           # L2 정규화
    random_state=42,
    eval_metric='logloss'
)

# 학습
start_time = time.time()
xgb_clf.fit(X_train, y_train)
train_time = time.time() - start_time

# 예측 및 평가
y_pred = xgb_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("=== XGBoost 분류 결과 ===")
print(f"훈련 정확도: {xgb_clf.score(X_train, y_train):.4f}")
print(f"테스트 정확도: {accuracy:.4f}")
print(f"학습 시간: {train_time:.4f}초")
print(f"\n분류 보고서:")
print(classification_report(y_test, y_pred, target_names=cancer.target_names))

### 2.2 조기 종료 (Early Stopping)

In [ ]:
# 검증 데이터 분리
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# 조기 종료 사용
xgb_early = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    early_stopping_rounds=10,  # 10 라운드 동안 개선 없으면 중지
    eval_metric='logloss'
)

xgb_early.fit(
    X_train_sub, y_train_sub,
    eval_set=[(X_val, y_val)],
    verbose=False
)

print("=== 조기 종료 결과 ===")
print(f"최적 반복 횟수: {xgb_early.best_iteration}")
print(f"최적 점수: {xgb_early.best_score:.4f}")
print(f"테스트 정확도: {xgb_early.score(X_test, y_test):.4f}")

### 2.3 특성 중요도

In [ ]:
# 특성 중요도 시각화
importance_types = ['weight', 'gain', 'cover']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, imp_type in zip(axes, importance_types):
    importance_dict = xgb_clf.get_booster().get_score(importance_type=imp_type)
    
    if importance_dict:
        # 상위 10개만 표시
        sorted_importance = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)[:10]
        features = [x[0] for x in sorted_importance]
        values = [x[1] for x in sorted_importance]
        
        ax.barh(range(len(features)), values)
        ax.set_yticks(range(len(features)))
        ax.set_yticklabels(features)
        ax.set_xlabel('Importance')
        ax.set_title(f'Feature Importance ({imp_type})')
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("""
중요도 타입:
- weight: 특성이 분할에 사용된 횟수
- gain: 특성 사용 시 평균 이득
- cover: 특성이 커버하는 평균 샘플 수
""")

### 2.4 XGBoost 하이퍼파라미터 튜닝

In [ ]:
# 파라미터 그리드
param_grid_xgb = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [100, 200],
    'min_child_weight': [1, 3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Grid Search (시간이 오래 걸릴 수 있으므로 간소화된 그리드 사용)
grid_search_xgb = GridSearchCV(
    XGBClassifier(random_state=42, eval_metric='logloss'),
    param_grid_xgb,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=0
)

# 시간 절약을 위해 샘플 사용
X_sample, _, y_sample, _ = train_test_split(X_train, y_train, train_size=0.3, random_state=42)
grid_search_xgb.fit(X_sample, y_sample)

print("=== XGBoost Grid Search 결과 ===")
print(f"최적 파라미터: {grid_search_xgb.best_params_}")
print(f"최적 CV 점수: {grid_search_xgb.best_score_:.4f}")
print(f"테스트 점수: {grid_search_xgb.score(X_test, y_test):.4f}")

## 3. LightGBM

In [ ]:
# LightGBM 설치: pip install lightgbm
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor

print("""
LightGBM 특징:

1. Leaf-wise 성장:
   - 기존: Level-wise (수평 분할)
   - LightGBM: Leaf-wise (손실 최대 감소 리프 분할)
   - 더 빠르고 정확하지만 과적합 위험

2. 히스토그램 기반 분할:
   - 연속형 값을 이산화
   - 메모리 효율적, 빠른 학습

3. GOSS (Gradient-based One-Side Sampling):
   - 그래디언트가 큰 샘플 위주로 샘플링

4. EFB (Exclusive Feature Bundling):
   - 상호 배타적 특성들을 묶음
   - 희소 특성에 효과적
""")

print(f"LightGBM 버전: {lgb.__version__}")

### 3.1 LightGBM 분류

In [ ]:
# LightGBM 분류기
lgb_clf = LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=-1,           # -1: 제한 없음
    num_leaves=31,          # 리프 노드 최대 수
    min_child_samples=20,   # 리프 노드 최소 샘플 수
    subsample=1.0,          # 행 샘플링
    colsample_bytree=1.0,   # 열 샘플링
    reg_alpha=0,            # L1 정규화
    reg_lambda=0,           # L2 정규화
    random_state=42,
    verbose=-1
)

# 학습
start_time = time.time()
lgb_clf.fit(X_train, y_train)
train_time_lgb = time.time() - start_time

# 평가
y_pred_lgb = lgb_clf.predict(X_test)
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)

print("=== LightGBM 분류 결과 ===")
print(f"훈련 정확도: {lgb_clf.score(X_train, y_train):.4f}")
print(f"테스트 정확도: {accuracy_lgb:.4f}")
print(f"학습 시간: {train_time_lgb:.4f}초")

### 3.2 num_leaves vs max_depth

In [ ]:
print("""
num_leaves와 max_depth의 관계:
- max_depth = d일 때, 최대 리프 수 = 2^d
- num_leaves = 31이면 대략 max_depth = 5 수준
- 과적합 방지: num_leaves < 2^max_depth

권장 설정:
- 대용량 데이터: num_leaves = 2^max_depth - 1 이하
- 소규모 데이터: num_leaves를 작게 (15~31)
""")

# num_leaves에 따른 성능
num_leaves_range = [15, 31, 63, 127, 255]
train_scores_lgb = []
test_scores_lgb = []

for num_leaves in num_leaves_range:
    lgb_temp = LGBMClassifier(
        n_estimators=100,
        num_leaves=num_leaves,
        random_state=42,
        verbose=-1
    )
    lgb_temp.fit(X_train, y_train)
    train_scores_lgb.append(lgb_temp.score(X_train, y_train))
    test_scores_lgb.append(lgb_temp.score(X_test, y_test))

plt.figure(figsize=(10, 6))
plt.plot(num_leaves_range, train_scores_lgb, 'o-', label='Train')
plt.plot(num_leaves_range, test_scores_lgb, 's-', label='Test')
plt.xlabel('num_leaves')
plt.ylabel('Accuracy')
plt.title('LightGBM: num_leaves Effect')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 3.3 특성 중요도

In [ ]:
# LightGBM 특성 중요도
importance_lgb = pd.DataFrame({
    'Feature': cancer.feature_names,
    'Importance': lgb_clf.feature_importances_
}).sort_values('Importance', ascending=True).tail(15)

plt.figure(figsize=(10, 8))
plt.barh(importance_lgb['Feature'], importance_lgb['Importance'])
plt.xlabel('Importance')
plt.title('LightGBM Feature Importance - Top 15')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. CatBoost 개요

In [ ]:
print("""
CatBoost 특징:

1. 범주형 특성 자동 처리:
   - Target Encoding 자동 적용
   - Ordered Target Statistics로 데이터 누수 방지

2. Ordered Boosting:
   - 학습 순서를 랜덤화하여 편향 감소
   - 과적합 방지

3. 대칭 트리:
   - 같은 수준의 모든 노드가 동일한 분할 조건 사용
   - 예측 속도 향상

설치: pip install catboost

기본 사용법:
from catboost import CatBoostClassifier

cat_clf = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    l2_leaf_reg=3,
    random_state=42,
    verbose=False
)

cat_clf.fit(X_train, y_train)
""")

## 5. 부스팅 알고리즘 비교

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# 모델 정의
models = {
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss'),
    'LightGBM': LGBMClassifier(n_estimators=100, random_state=42, verbose=-1)
}

# 비교
print("부스팅 알고리즘 비교:")
print("-" * 70)
print(f"{'모델':<20} {'훈련 정확도':>15} {'테스트 정확도':>15} {'학습시간(초)':>15}")
print("-" * 70)

results = {}
for name, model in models.items():
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    
    results[name] = {
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'time': train_time
    }
    
    print(f"{name:<20} {train_acc:>15.4f} {test_acc:>15.4f} {train_time:>15.4f}")

print("-" * 70)

In [ ]:
# 시각화 비교
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 정확도 비교
names = list(results.keys())
test_accuracies = [results[n]['test_accuracy'] for n in names]
axes[0].barh(names, test_accuracies, color=['skyblue', 'salmon', 'lightgreen'])
axes[0].set_xlabel('Test Accuracy')
axes[0].set_title('Accuracy Comparison')
axes[0].set_xlim([0.9, 1.0])
axes[0].grid(True, alpha=0.3)

# 학습 시간 비교
times = [results[n]['time'] for n in names]
axes[1].barh(names, times, color=['skyblue', 'salmon', 'lightgreen'])
axes[1].set_xlabel('Training Time (seconds)')
axes[1].set_title('Training Time Comparison')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 회귀 예제

In [ ]:
# California Housing 데이터
housing = fetch_california_housing()
X_reg, y_reg = housing.data, housing.target

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

print(f"데이터 크기: {X_reg.shape}")
print(f"특성: {housing.feature_names}")
print(f"타겟: Median house value (in $100,000s)")

In [ ]:
# XGBoost 회귀
xgb_reg = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)
xgb_reg.fit(X_train_reg, y_train_reg)
y_pred_xgb_reg = xgb_reg.predict(X_test_reg)

# LightGBM 회귀
lgb_reg = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    num_leaves=31,
    random_state=42,
    verbose=-1
)
lgb_reg.fit(X_train_reg, y_train_reg)
y_pred_lgb_reg = lgb_reg.predict(X_test_reg)

# 평가
print("=== XGBoost 회귀 ===")
print(f"R² Score: {r2_score(y_test_reg, y_pred_xgb_reg):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_reg, y_pred_xgb_reg)):.4f}")

print("\n=== LightGBM 회귀 ===")
print(f"R² Score: {r2_score(y_test_reg, y_pred_lgb_reg):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_reg, y_pred_lgb_reg)):.4f}")

In [ ]:
# 예측 vs 실제 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# XGBoost
axes[0].scatter(y_test_reg, y_pred_xgb_reg, alpha=0.5)
axes[0].plot([y_test_reg.min(), y_test_reg.max()], 
             [y_test_reg.min(), y_test_reg.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual')
axes[0].set_ylabel('Predicted')
axes[0].set_title(f'XGBoost (R²={r2_score(y_test_reg, y_pred_xgb_reg):.4f})')
axes[0].grid(True, alpha=0.3)

# LightGBM
axes[1].scatter(y_test_reg, y_pred_lgb_reg, alpha=0.5, color='green')
axes[1].plot([y_test_reg.min(), y_test_reg.max()], 
             [y_test_reg.min(), y_test_reg.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual')
axes[1].set_ylabel('Predicted')
axes[1].set_title(f'LightGBM (R²={r2_score(y_test_reg, y_pred_lgb_reg):.4f})')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 하이퍼파라미터 가이드

In [ ]:
# 하이퍼파라미터 비교표
params_comparison = pd.DataFrame({
    'Parameter': ['학습률', '트리 수', '깊이', '리프 수', 'L1 정규화', 'L2 정규화', '행 샘플링', '열 샘플링'],
    'XGBoost': ['learning_rate', 'n_estimators', 'max_depth', '-', 'reg_alpha', 'reg_lambda', 'subsample', 'colsample_bytree'],
    'LightGBM': ['learning_rate', 'n_estimators', 'max_depth', 'num_leaves', 'reg_alpha', 'reg_lambda', 'subsample', 'colsample_bytree'],
    'Effect': ['낮으면 안정적', '많으면 정확', '깊으면 복잡', '많으면 복잡', '과적합 방지', '과적합 방지', '분산 감소', '다양성 증가']
})

print("하이퍼파라미터 가이드:")
print(params_comparison.to_string(index=False))

In [ ]:
print("""
권장 튜닝 순서:

1. 트리 구조 파라미터:
   - max_depth, num_leaves
   - min_child_weight, min_child_samples

2. 샘플링 파라미터:
   - subsample
   - colsample_bytree

3. 정규화 파라미터:
   - reg_alpha, reg_lambda

4. 학습률 조정:
   - learning_rate 낮추고
   - n_estimators 늘리기

과적합 방지 전략:
- 조기 종료 (early_stopping_rounds)
- 정규화 (reg_alpha, reg_lambda)
- 샘플링 (subsample, colsample_bytree)
- 트리 제한 (max_depth, min_child_weight)
- 학습률 낮추기 (learning_rate)
""")

## 정리

### 알고리즘 비교

| 알고리즘 | 특징 | 장점 | 단점 |
|----------|------|------|------|
| Gradient Boosting | 잔차 학습 | 높은 정확도 | 느린 학습 |
| XGBoost | 정규화 + 병렬화 | 빠름, 정확함 | 메모리 사용 |
| LightGBM | Leaf-wise | 매우 빠름, 대용량 | 과적합 위험 |
| CatBoost | 범주형 처리 | 튜닝 적게 필요 | 느린 시작 |

### 선택 가이드

- **작은 데이터 (<10K)**: XGBoost 또는 sklearn GradientBoosting
- **중간 데이터 (10K-100K)**: XGBoost
- **대용량 데이터 (>100K)**: LightGBM
- **범주형 특성 많음**: CatBoost
- **빠른 학습 필요**: LightGBM
- **최고 정확도**: 모두 시도 후 앙상블

### 주요 하이퍼파라미터

**공통:**
- `n_estimators`: 트리 개수
- `learning_rate`: 학습률
- `max_depth`: 트리 깊이

**XGBoost 전용:**
- `min_child_weight`: 리프 노드 최소 가중치
- `gamma`: 분할 최소 손실 감소

**LightGBM 전용:**
- `num_leaves`: 리프 노드 최대 수
- `min_child_samples`: 리프 노드 최소 샘플

### 다음 단계
- Stacking과 Blending
- AutoML (Optuna, Hyperopt)
- 실전 Kaggle 대회 참여